In [2]:
pip install pyLDAvis

     |████████████████████████████████| 1.7MB 7.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.3MB 189kB/s 
     |████████████████████████████████| 9.9MB 37.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136870 sha256=792fb7ea5960437e5b580f021a6fafe823d4319ef329f47b2d87b61d4f45c464
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0

In [10]:
#Dependencies
import pandas as pd
import gensim #the library for Topic modelling
from gensim import corpora, models
from nltk.corpus import stopwords
import os
import string
from nltk.stem.wordnet import WordNetLemmatizer
import pickle
import warnings
warnings.simplefilter('ignore')
from itertools import chain

In [6]:
os.getcwd()
os.chdir('drive/MyDrive/NLP')
os.mkdir('webtranscripts')


In [50]:

import requests
from bs4 import BeautifulSoup
  
URL = ["https://developers.google.com/search/docs/beginner/seo-starter-guide",
       'https://www.brandpoint.com/blog/examples-of-high-quality-content-seo/',
       'https://www.wordstream.com/blog/ws/2015/04/30/seo-basics',
       'https://www.reliablesoft.net/seo-campaign/',
       'https://www.bruceclay.com/blog/what-is-seo-article/#:~:text=An%20SEO%20article%20is%20a,page%20with%20a%20unique%20URL.&text=An%20SEO%20article%20is%20also,1%20of%20the%20search%20results.',
       'https://blog.jdrgroup.co.uk/digital-prosperity-blog/what-are-seo-articles',
       'https://neilpatel.com/blog/seo-copywriting/'
       ]
websites = ['developers', "brandpoint", 'wordstream', 'reliablesoft', 'bruceclay', 'jdrgroup', 'neilpatel1']
def url_to_script(URL):
  r = requests.get(URL)
  soup = BeautifulSoup(r.content, "html5lib") 
  text = [p.text for p in soup.find_all('p')]
  print(URL)
  return text

In [51]:
transcripts = [url_to_script(u) for u in URL]

for i, c in enumerate(websites):
    with open("webtranscripts/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts[i], file)

https://developers.google.com/search/docs/beginner/seo-starter-guide
https://www.brandpoint.com/blog/examples-of-high-quality-content-seo/
https://www.wordstream.com/blog/ws/2015/04/30/seo-basics
https://www.reliablesoft.net/seo-campaign/
https://www.bruceclay.com/blog/what-is-seo-article/#:~:text=An%20SEO%20article%20is%20a,page%20with%20a%20unique%20URL.&text=An%20SEO%20article%20is%20also,1%20of%20the%20search%20results.
https://blog.jdrgroup.co.uk/digital-prosperity-blog/what-are-seo-articles
https://neilpatel.com/blog/seo-copywriting/


In [52]:

# Load pickled files
data = {}
for i, c in enumerate(websites):
    with open("webtranscripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [53]:
data.keys()

dict_keys(['developers', 'brandpoint', 'wordstream', 'reliablesoft', 'bruceclay', 'jdrgroup', 'neilpatel1'])

In [ ]:
data['jdrgroup']

In [57]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [58]:
# Combine it!
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [ ]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

In [60]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

In [62]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

In [64]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [65]:
#clean the data
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

In [66]:
def clean_text_round3(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    normalized = ' '.join([lemma.lemmatize(word) for word in stop_free.split()])
    return normalized

round3 = lambda x: clean_text_round3(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round3))
data_clean

In [68]:
# Let's pickle it for later use
data_df.to_pickle("webtranscripts/corpus.pkl")
data_clean.to_pickle("webtranscripts/clean.pkl")

In [ ]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

In [71]:
# Let's pickle it for later use
data_dtm.to_pickle("webtranscripts/dtm.pkl")

In [72]:
pickle.dump(cv, open("webtranscripts/cv.pkl", "wb"))